In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.conf import SparkConf
config = SparkConf()
# config.set("property", "value")
config.setMaster("local").setAppName("HelloSparkSession")

from pyspark.sql import SparkSession
# spark Session, entry point for Spark SQL, DataFrame
# in single spark driver/note book/spark application,
# there can be many spark sessions, and 
# only one spark context
spark = SparkSession.builder\
                    .config(conf=config)\
                    .getOrCreate()
# spark core operations, like rdd, partitions, actions etc
# spark session shall use catalyst engine, which will use spark context for low level
# code execution
sc = spark.sparkContext

22/05/17 22:05:14 WARN Utils: Your hostname, ubuntu-virtual-machine resolves to a loopback address: 127.0.1.1; using 192.168.174.129 instead (on interface ens33)
22/05/17 22:05:14 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/05/17 22:05:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
# Databricks notebook source
# Databricks notebook source
products = [ 
          # (product_id, product_name, brand_id)  
         (1, 'iPhone', 100),
         (2, 'Galaxy', 200),
         (3, 'Redme', 300), #   no matching brand
         (4, 'Pixel', 400),
]

brands = [
    #(brand_id, brand_name)
    (100, "Apple"),
    (200, "Samsung"),
    (400, "Google"),
    (500, "Sony"), # no matching products
]



productDf = spark.createDataFrame(data=products, schema=["product_id", "product_name", "brand_id"])
productDf.show()

brandDf = spark.createDataFrame(data=brands, schema=["brand_id", "brand_name"])

brandDf.show()

store = [
    #(store_id, store_name)
    (1000, "Poorvika"),
    (2000, "Sangeetha"),
    (4000, "Amazon"),
    (5000, "FlipKart"), 
]
 
storeDf = spark.createDataFrame(data=store, schema=["store_id", "store_name"])
storeDf.show()

+----------+------------+--------+
|product_id|product_name|brand_id|
+----------+------------+--------+
|         1|      iPhone|     100|
|         2|      Galaxy|     200|
|         3|       Redme|     300|
|         4|       Pixel|     400|
+----------+------------+--------+

+--------+----------+
|brand_id|brand_name|
+--------+----------+
|     100|     Apple|
|     200|   Samsung|
|     400|    Google|
|     500|      Sony|
+--------+----------+

+--------+----------+
|store_id|store_name|
+--------+----------+
|    1000|  Poorvika|
|    2000| Sangeetha|
|    4000|    Amazon|
|    5000|  FlipKart|
+--------+----------+



In [4]:
# in any spark application, there will be ONLY ONE spark context
# and as many spark sessions allowed


spark2 = spark.newSession()

In [5]:
spark  

In [6]:
spark2

In [7]:
# create product temp table in spark session
productDf.createOrReplaceTempView("products")


In [8]:
# now access products from session /knew it will work
spark.sql("SELECT * FROM products").show()

+----------+------------+--------+
|product_id|product_name|brand_id|
+----------+------------+--------+
|         1|      iPhone|     100|
|         2|      Galaxy|     200|
|         3|       Redme|     300|
|         4|       Pixel|     400|
+----------+------------+--------+



In [9]:
# now try to access products from spark2, IT WILL FAIL, as products table private to spark session
spark2.sql("SELECT * FROM products").show() # error  AnalysisException: Table or view not found: products; 

AnalysisException: Table or view not found: products; line 1 pos 14;
'Project [*]
+- 'UnresolvedRelation [products], [], false


In [10]:
# now create global temp view global_temp that can be shared across multiple sessions on same notebook

brandDf.createOrReplaceGlobalTempView ("brands")

spark.sql("SHOW TABLES IN global_temp").show()

+-----------+---------+-----------+
|   database|tableName|isTemporary|
+-----------+---------+-----------+
|global_temp|   brands|       true|
|           | products|       true|
+-----------+---------+-----------+



In [11]:
spark.sql("SELECT * FROM global_temp.brands").show()

+--------+----------+
|brand_id|brand_name|
+--------+----------+
|     100|     Apple|
|     200|   Samsung|
|     400|    Google|
|     500|      Sony|
+--------+----------+

